<a href="https://colab.research.google.com/github/lephuocvinhlinh/-AI_City2021-Scence_Text_Recognition/blob/main/%5BLPVL%5DPaddle_OCR_SceneTextDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nhớ tạo một bản copy trước khi chạy

In [ ]:
!nvidia-smi

Fri Aug 13 00:58:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Clone source code và set-up tài nguyên

## Git clone và setup môi trường

In [ ]:
# Cài đặt paddlepaddle-gpu
!python3 -m pip install paddlepaddle-gpu==2.1.1 -f https://paddlepaddle.org.cn/whl/mkl/stable.html

In [ ]:
# Clone source code
%cd /content/
!git clone https://github.com/thanhnhan311201/PaddleOCR.git

In [ ]:
# Cài đặt các thư viện cần thiết
%cd /content/PaddleOCR
!python3 -m pip install -r requirements.txt

## Tải pretrain_models

In [ ]:
# Tạo thư mục để chứa pretrain model
%cd /content/PaddleOCR/
!mkdir pretrain_models

In [ ]:
# Tải pretrain model
%cd /content/PaddleOCR/pretrain_models/
!wget -c https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_r50_vd_sast_icdar15_v2.0_train.tar

In [ ]:
# Giải nén pretrain model
%cd /content/PaddleOCR/pretrain_models/
!tar -xf det_r50_vd_sast_icdar15_v2.0_train.tar

# Chuẩn bị train_data và eval_data

In [ ]:
# Tải bộ data train và eval đã được chuẩn bị sẵn
%cd /content/PaddleOCR/
!gdown --id 1HpdyKFIHSNLJMxxbFcqNarGhrXDc7jb7 -O train_data.zip
!gdown --id 11dAAt66pKh3jGAig3t31PGXvnCAo4jyV -O eval_data.zip

In [ ]:
# Giải nén data train và eval
%cd /content/PaddleOCR/
!unzip train_data.zip
!unzip eval_data.zip

## Tải checkpoint

In [ ]:
# Tạo thư mục để chứa file checkpoints
%cd /content/PaddleOCR/
!mkdir output/
%cd output/
!mkdir sast_r50_vd_ic15

In [ ]:
# Tải về file checkpoints
%cd /content/PaddleOCR/output/sast_r50_vd_ic15/
!gdown --id 1DNlyY-zL_2aLk4YjKDnDcgpzfblLZ285
!gdown --id 1DNHkzyse-z5q1yKhFgMO6PzN4ey5O6ea
!gdown --id 1DQsI3xqkpXrMb0N1BlMcWDGWECFINAqi

# Train

In [ ]:
%cd /content/PaddleOCR/
# Nếu muốn train từ checkpoints đã train thì thêm vào: -o Global.checkpoints=./output/sast_r50_vd_ic15/best_accuracy , mặc định trong file config là train từ best_accuracy của pretrain model
# Nếu muốn thay đổi bộ dữ liệu train và eval thì vô file config trong folder configs/det/det_r50_vd_sast_icdar15_fixed.yml, sửa lại path trong data_dir và label_file_list của phần Train và Eval
!python tools/train.py -c configs/det/det_r50_vd_sast_icdar15_fixed.yml -o Optimizer.base_lr=0.0001

# Test

## Tải test data

In [ ]:
# Tải về data public_test
%cd /content/PaddleOCR/
!gdown --id 1t_Lb4jVpIDPdyvgSds9lcl62UC8xbaM5 -O public_test.zip
!unzip public_test.zip

## Test

In [ ]:
%cd /content/PaddleOCR/
!python3 tools/export_model.py -c configs/det/det_r50_vd_sast_icdar15_fixed.yml \
       -o Global.checkpoints=/content/PaddleOCR/output/sast_r50_vd_ic15/best_accuracy \
       Global.save_inference_dir=/content/PaddleOCR/inference/sast_r50_vd_ic15/

In [ ]:
%cd /content/PaddleOCR/
# Nếu muốn thay đổi bộ dữ liệu test thì sửa lại path ở --image_dir tới folder dữ liệu test
!python3 tools/infer/predict_det_fixed.py --det_algorithm="SAST" \
  --image_dir=/content/PaddleOCR/public_test/ \
  --det_model_dir=/content/PaddleOCR/inference/sast_r50_vd_ic15/

## Chuyển định dạng file submit

In [ ]:
import os
%cd /content/PaddleOCR/inference_results/
directory = os.listdir('/content/PaddleOCR/inference_results/')
count = 0
text_file = open("aaaa.txt","a")
for file in directory:
	if file.endswith(".txt"):
		count = count + 1
		with open(file) as f:
			content = f.readlines()
	# you may also want to remove whitespace characters like `\n` at the end of each line
		content = [x.strip() for x in content] 
		for i in range(len(content)) :
			text_file.write("{0},{1}\n".format(file,content[i]))
   
print(count)

In [ ]:
# Tạo folder result để chứa file submission
%cd /content/PaddleOCR/
!mkdir result

In [ ]:
%cd /content/PaddleOCR/result/
with open('/content/PaddleOCR/inference_results/aaaa.txt') as f:
  contents = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
contents = [x.strip() for x in contents] 

for i in range(len(contents)):
  line = contents[i].split(',')
  file_name = line[0]
  line[0] = file_name[4:-4] + ".jpg"
  text_file = open('test.txt', "a")
  text_file.write("{0},{1},{2},{3},{4},{5},{6},{7},{8}\n".format(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8]))

# Sau khi chạy xong đoạn code này thì file submission nằm trong folder result mới tạo

# Submission

In [ ]:
import math, requests

def submit_prediction(df, comment='', **kwargs):
    TOKEN='f9f0fe020c715fcc74d65e919507405f1ff39dddfd93556c23513e77abb5b6dbe478fc7a6d00f2fc78f1b138fd0f9338b4e485db6e7406c02954bc92e2c7b0a2'
    URL='http://submission.mmlab.uit.edu.vn/api/submissions'
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': open(df, 'rb')},data={'comment':comment})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)
        
submit_prediction('/content/PaddleOCR/result/test.txt', comment='my submission')